In [7]:
import numpy as np
from tqdm import tqdm
import glob
from pathlib import Path
import os
fnames = glob.glob(os.path.join("/work/ds2/data/k.witham/CSPB.ML.2018R2_nonoise", "extracted_signals", "*", "signal*.tim"))
fnames = sorted(fnames, key=lambda x: int(Path(x).stem.split("_")[-1]))

x = np.array([np.fromfile(fname)[1:].view(np.complex64) for fname in tqdm(fnames)])


100%|██████████| 112000/112000 [22:01<00:00, 84.78it/s] 


In [8]:
import os
import pandas as pd
labels = pd.read_csv(os.path.join("/work/ds2/data/k.witham/CSPB.ML.2018R2_nonoise", "signal_record.txt"), header=None, sep='\s+', index_col=0, names=['label', 'T0', 'cfo', 'beta', 'U', 'D', 'SNR', 'P(N)'])
labels

,label,T0,cfo,beta,U,D,SNR,P(N)
1,bpsk,2,-0.000839,0.824908,7,6,2.566627,-1000.0
2,qpsk,2,-0.000839,0.824908,7,6,2.566627,-1000.0
3,8psk,2,-0.000839,0.824908,7,6,2.566627,-1000.0
4,dqpsk,2,-0.000839,0.824908,7,6,2.566627,-1000.0
5,msk,2,-0.000839,0.824908,7,6,2.566627,-1000.0
...,...,...,...,...,...,...,...,...
111996,dqpsk,9,-0.000046,0.564858,9,8,3.255398,-1000.0
111997,msk,9,-0.000046,0.564858,9,8,3.255398,-1000.0
111998,16qam,9,-0.000046,0.564858,9,8,3.255398,-1000.0
111999,64qam,9,-0.000046,0.564858,9,8,3.255398,-1000.0


In [9]:
classes = ['bpsk', 'qpsk', '8psk', 'dqpsk', 'msk', '16qam', '64qam', '256qam']
y = np.array([classes.index(l) for l in labels.label])

In [10]:
# Set all MSK to a beta of 0.5, this gives us about null-to-null for bandwidth
labels.loc[labels['label'] == "msk", 'beta'] = 0.5

In [11]:
t0 = labels.T0.to_numpy()
d = labels.D.to_numpy()
u = labels.U.to_numpy()
f_sym = (1/t0)*(d/u)
T_s = 1/f_sym
# f_sym[d==0] = (1/t0[d==0])

In [12]:
import h5py
h5py_path = "/work/ds2/data/k.witham/CSPB.ML.2018R2_nonoise/CSPB.ML.2018R2_nonoise.hdf5"

with h5py.File(h5py_path, "w") as f:
    f['x'] = x
    f['y'] = y
    f['T0'] = labels.T0.to_numpy()
    f['cfo'] = labels.cfo.to_numpy()
    f['beta'] = labels.beta.to_numpy()
    f['U'] = labels.U.to_numpy()
    f['D'] = labels.D.to_numpy()
    f['T_s'] = T_s